In [2]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [11]:
df = pd.read_csv('../dataset.csv')
df =df.sample(n=10000)

In [12]:
df.head()

,text,label
3003,Banjir setinggi 2.5 meter melanda Medan. 54 RT...,banjir
5546,Tanah longsor di Agam pada 2023-10-10. 1 rumah...,tanah_longsor
6388,Peringatan tsunami di Banda Aceh setelah gempa...,tsunami
4926,Tanah longsor di Puncak pada 2021-09-15. 25 ru...,tanah_longsor
5133,Tanah longsor di Pacitan pada 2024-11-19. 11 r...,tanah_longsor


In [13]:
df['label'].value_counts()

label
banjir           2000
tanah_longsor    2000
tsunami          2000
gempabumi        2000
not_relevant     2000
Name: count, dtype: int64